In [1]:
from proofnet import cryptography
from proofnet import protocol #import Store, Retrieve

In [2]:
def store( synapse: protocol.Store ) -> protocol.Store:
    # Check content_hash against the content
    local_content_hash = cryptography.hash( synapse.content )
    bt.logging.debug(f"Synapse content: {synapse.content}")
    bt.logging.debug(f"Synapse content hash: {synapse.content_hash}")
    bt.logging.debug(f"Local content hash: {local_content_hash}")
    stored = False
    try:
        # If it matches, check the signature against the pubkey
        if synapse.content_hash == local_content_hash:
            bt.logging.debug(f"Local content hash matches synapse content hash.")
            if cryptography.verify( synapse.content, synapse.signature, synapse.pubkey ):
                # # If it matches, generate a signature of the content signed with the miner key
                # store the content has as key and the (miner_signature, pubkey) pairs in the database
                miner_signature, miner_pubkey = cryptography.sign_content_with_new_keypair( synapse.content_hash )
                self.registry[ synapse.content_hash ] = ( miner_signature, miner_pubkey )
                bt.logging.debug(f"Stored content hash: {synapse.content_hash}")
                bt.logging.debug(f"Stored miner signature: {miner_signature}")
                bt.logging.debug(f"Stored miner pubkey: {miner_pubkey}")
                stored = True
                # Optimistically store (no need to send back the signature until verify step)
            else:
                # If it doesn't match, return an error. Attempted to store invalid content.
                stored = False
                raise SignatureMismatchError( "Signature is not valid with provided pubkey!" )
        else:
            # If it doesn't match, return an error.
            stored = False
            raise ContentHashMismatchError( "Content hash mismatch, data tampered with!" )
    except SignatureMismatchError as e:
        synapse.error_message = e
    except ContentHashMismatchError as e:
        synapse.error_message = e
    except Exception as e:
        synapse.error_message = "Unknown error occured."
    finally:
        # return the filled synapse
        synapse.stored = stored
        return synapse

def retrieve( synapse: protocol.Retrieve ) -> protocol.Retrieve:
    registry_indices = synapse.registry_indices ^ len( self.registry ) % len( registry )
    hashes = np.asarray(list(self.registry))
    miner_data = {}
    for index in registry_indices:
        # Attempt to extract the hash from the registry (if it exists)
        try:
            hash_i = hashes[index]
        except:
            miner_data[ hash_i ] = ( None, None ) # not found in registry.
            continue
        # Retrive the miner signature and pubkey from the registry
        miner_signature, miner_pubkey = self.registry[ hash_i ]
        miner_data[ hash_i ] = ( miner_signature, miner_pubkey )
    # Fill the synapse with the miner signatures and pubkeys
    synapse.miner_data = miner_data
    # return the filled synapse
    return synapse

In [3]:
content = "Hello, world!"
signature, pubkey = cryptography.sign_content_with_new_keypair( content )
content_hash = cryptography.hash( content )
syn = protocol.Store( 
    content=content, content_hash=content_hash, pubkey=pubkey, signature=signature
)
syn

/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/constants.py:192: CryptographyDeprecationWarning: IDEA has been deprecated
  bs = {SymmetricKeyAlgorithm.IDEA: algorithms.IDEA,
/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/constants.py:194: CryptographyDeprecationWarning: CAST5 has been deprecated
  SymmetricKeyAlgorithm.CAST5: algorithms.CAST5,
/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/constants.py:195: CryptographyDeprecationWarning: Blowfish has been deprecated
  SymmetricKeyAlgorithm.Blowfish: algorithms.Blowfish,


Store(content='Hello, world!', content_hash='315f5bdb76d078c43b8ac0064e4a0164612b1fce77c869345bfc94c75894edd3', pubkey='-----BEGIN PGP PUBLIC KEY BLOCK-----\n\nxsFNBGUCfJYBEAC/yduqWBH9y/hoHQ+b6zNB17PrxGZ2ym1d3d6AiAvmK3EDQrOR\nlRD+PH3wSzmLxC5jSVcyXaIDpunYTJ2QfpUAzBohihCOyzzFUvBc8pkxppa0tFzU\nqVKWgHrni4x6sKcujeQGSB8EC2MpZ5c4Cj4osynmpo8zTlLkJyDhRpulKxK98lNK\nJxhXq65k2inc0iixEw+nT8W1CfEwVCoalCaPPDUpcQsDZcTib+6gdXb4pGKPPkb/\nR9h7k9RPTlP64RDNJQx5dkopUbDTZW8JdF0q8Qskfgm6C1M9O0b/rTua4JEUuZYS\nqQwEm6/gPey6bcJ8TzqD3YtiPv8A7dxo91jCWGua5emgu9FevV36bcdxAdQ7TLqc\nwkujw1yRTnvnTFbHpvVZdoZecJxNnyIN9IEHmu+9eQVctE9hORg/19hm0pamE7Hp\ne+FfNPUDbP8T5qK+MTDpOQlq4WRm4t2zInZ53S/ZQfeXqQcL9kzVautY+DI8PIqx\nJcWf59dpIrkJLubVaC4miQjc63SjuzRc7O8lwZEIPI2Pl/pUolXHk6dGA/y5+oY7\nho6ad6s1EjVf5htsOlUXfLTeoXb3ikGkFlwAZktFsvWFgi46F8Airq06O0fxpL6n\n6RFaBwC5rYVxv8DVvIXM1elHj/s7+mE+fWF0JPym5STditJhJFp105z9TQARAQAB\nzRxUZXN0IFVzZXIgPHRlc3RAZXhhbXBsZS5jb20+wsGGBBMBCAAwBQJlAnyWAhsO\nAwsJBwMVCAoEFgIBAAIeARYhBII46zJpfSNgsVtPHWqFpVro

In [4]:
import bittensor as bt
bt.trace()
axon = bt.axon( port=9123 )

axon.attach(
    forward_fn = store
).attach(
    forward_fn = retrieve
)
axon.start()

Axon([::], 9123, 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH, started, ['Synapse', 'Store', 'Retrieve'])

INFO:     Started server process [519863]
INFO:     Waiting for application startup.
TRACE:    ASGI [1] Started scope={'type': 'lifespan', 'asgi': {'version': '3.0', 'spec_version': '2.0'}, 'state': {}}
TRACE:    ASGI [1] Receive {'type': 'lifespan.startup'}
TRACE:    ASGI [1] Send {'type': 'lifespan.startup.complete'}
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9123 (Press CTRL+C to quit)


In [5]:
# Create a Dendrite instance to handle client-side communication.
d = bt.dendrite()

# Send a request to the Axon using the Dendrite, passing in a StreamPrompting instance with roles and messages.
# The response is awaited, as the Dendrite communicates asynchronously with the Axon.
resp = await d(
    [axon],
    syn
)

# The response object contains the result of the streaming operation.
resp[0]

2023-09-14 03:23:07.980 |      TRACE       | Pre-process synapse for request


2023-09-14 03:23:08.030 |      DEBUG       | dendrite | --> | 6243 B | Store | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 216.153.62.113:9123 | 0 | Success


TRACE:    127.0.0.1:42100 - HTTP connection made
TRACE:    127.0.0.1:42100 - ASGI [2] Started scope={'type': 'http', 'asgi': {'version': '3.0', 'spec_version': '2.3'}, 'http_version': '1.1', 'server': ('127.0.0.1', 9123), 'client': ('127.0.0.1', 42100), 'scheme': 'http', 'method': 'POST', 'root_path': '', 'path': '/Store', 'raw_path': b'/Store', 'query_string': b'', 'headers': '<...>', 'state': {}}


2023-09-14 03:23:08.042 |      DEBUG       | axon     | <-- | 3464 B | Store | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 127.0.0.1:42100 | 200 | Success 
2023-09-14 03:23:08.042 |      TRACE       | Check verification            


TRACE:    127.0.0.1:42100 - ASGI [2] Receive {'type': 'http.request', 'body': '<3464 bytes>', 'more_body': False}


...

TRACE:    127.0.0.1:42100 - ASGI [2] Completed


2023-09-14 03:23:09.615 |      DEBUG       | dendrite | <-- | 6734 B | Store | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 216.153.62.113:9123 | 200 | Success


Store(content='Hello, world!', content_hash='315f5bdb76d078c43b8ac0064e4a0164612b1fce77c869345bfc94c75894edd3', pubkey='-----BEGIN PGP PUBLIC KEY BLOCK-----\n\nxsFNBGUCfJYBEAC/yduqWBH9y/hoHQ+b6zNB17PrxGZ2ym1d3d6AiAvmK3EDQrOR\nlRD+PH3wSzmLxC5jSVcyXaIDpunYTJ2QfpUAzBohihCOyzzFUvBc8pkxppa0tFzU\nqVKWgHrni4x6sKcujeQGSB8EC2MpZ5c4Cj4osynmpo8zTlLkJyDhRpulKxK98lNK\nJxhXq65k2inc0iixEw+nT8W1CfEwVCoalCaPPDUpcQsDZcTib+6gdXb4pGKPPkb/\nR9h7k9RPTlP64RDNJQx5dkopUbDTZW8JdF0q8Qskfgm6C1M9O0b/rTua4JEUuZYS\nqQwEm6/gPey6bcJ8TzqD3YtiPv8A7dxo91jCWGua5emgu9FevV36bcdxAdQ7TLqc\nwkujw1yRTnvnTFbHpvVZdoZecJxNnyIN9IEHmu+9eQVctE9hORg/19hm0pamE7Hp\ne+FfNPUDbP8T5qK+MTDpOQlq4WRm4t2zInZ53S/ZQfeXqQcL9kzVautY+DI8PIqx\nJcWf59dpIrkJLubVaC4miQjc63SjuzRc7O8lwZEIPI2Pl/pUolXHk6dGA/y5+oY7\nho6ad6s1EjVf5htsOlUXfLTeoXb3ikGkFlwAZktFsvWFgi46F8Airq06O0fxpL6n\n6RFaBwC5rYVxv8DVvIXM1elHj/s7+mE+fWF0JPym5STditJhJFp105z9TQARAQAB\nzRxUZXN0IFVzZXIgPHRlc3RAZXhhbXBsZS5jb20+wsGGBBMBCAAwBQJlAnyWAhsO\nAwsJBwMVCAoEFgIBAAIeARYhBII46zJpfSNgsVtPHWqFpVro

2023-09-14 03:23:09.615 |      DEBUG       | dendrite | <-- | 6734 B | Store | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 216.153.62.113:9123 | 200 | Success


TRACE:    127.0.0.1:42100 - HTTP connection lost


In [8]:
resp[0].stored

False

In [6]:
ret = Retrieve(random_indices=[0])
ret

TRACE:    127.0.0.1:57658 - ASGI [2] Completed


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 ret = Retrieve(random_indices=[0])                                                           │
│   2 ret                                                                                          │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'Retrieve' is not defined

TRACE:    127.0.0.1:57658 - HTTP connection lost
